*This notebook will create a file with the following transformations/filters:*

- Filtered to include only the following columns: ('SR_NUMBER', 'SR_TYPE', 'OWNER_DEPARTMENT',
  'STATUS', 'CREATED_DATE', 'CLOSED_DATE', 'DUPLICATE', 'PARENT_SR_NUMBER', 'COMMUNITY_AREA',
  'WARD', 'CREATED_HOUR', 'CREATED_DAY_OF_WEEK', 'CREATED_MONTH')
- 311 Information-Only calls removed
- Legacy records removed
- Removed rows with no Ward or Community Area
- Added 'time_to_close_sec' column including time difference in seconds between record creation
  and closure

**Transformations/filters remaining:**

- Add 'number of children' column denoting how many duplicates a request has
- Filter out duplicates once first bullet point completed (this code is already in the notebook, we'll just need to move it and uncomment it)
- Create dummy columns for request type, department, community area, ward, and hour/day/month of request creation
- Think about what our 'time-to-close' threshold should be and apply that filter -- should we filter out all requests closed in under X minutes? Or should we filter out a request type entirely if more than X% of its requests are filled in under X minutes?

**Notes/resources**:

- pandas to_pickle function: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_pickle.html
- pandas read_pickle function: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_pickle.html#pandas.read_pickle
- I downloaded the Chicago 311 CSV file locally from the data portal: https://data.cityofchicago.org/Service-Requests/311-Service-Requests/v6vf-nfxy; it's ~1.4 GBs. I have it in a folder called 'raw_data' in my local repo (I didn't push it because the file is too big). You'll need to do the same on your end (download the file and put it in a 'raw_data' folder in your local repo). Let me know if you have any trouble with this (e.g., you don't have space to save a 1.4 GB file or it takes too long to download)
- I don't think we should push another pickle file to GitHub until the final version, but open to what you think. I think we can add the final filters/transformations to this notebook.

In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
import datetime as dt

Read in downloaded CSV file:

In [3]:
chi_311 = pd.read_csv('../raw_data/chicago_311_requests.csv')

/Users/michaelfeldman/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (11,12,18,19,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
chi_311.shape

(4080622, 37)

In [5]:
chi_311.columns

Index(['SR_NUMBER', 'SR_TYPE', 'SR_SHORT_CODE', 'OWNER_DEPARTMENT', 'STATUS',
       'CREATED_DATE', 'LAST_MODIFIED_DATE', 'CLOSED_DATE', 'STREET_ADDRESS',
       'CITY', 'STATE', 'ZIP_CODE', 'STREET_NUMBER', 'STREET_DIRECTION',
       'STREET_NAME', 'STREET_TYPE', 'DUPLICATE', 'LEGACY_RECORD',
       'LEGACY_SR_NUMBER', 'PARENT_SR_NUMBER', 'COMMUNITY_AREA', 'WARD',
       'ELECTRICAL_DISTRICT', 'ELECTRICITY_GRID', 'POLICE_SECTOR',
       'POLICE_DISTRICT', 'POLICE_BEAT', 'PRECINCT',
       'SANITATION_DIVISION_DAYS', 'CREATED_HOUR', 'CREATED_DAY_OF_WEEK',
       'CREATED_MONTH', 'X_COORDINATE', 'Y_COORDINATE', 'LATITUDE',
       'LONGITUDE', 'LOCATION'],
      dtype='object')

Filter dataframe to exclude info-only calls:

In [6]:
chi_311_filtered = chi_311[chi_311['SR_TYPE'] != '311 INFORMATION ONLY CALL']

In [7]:
chi_311_filtered.shape

(2702596, 37)

Filter for columns needed:

In [8]:
chi_311_filtered.columns

Index(['SR_NUMBER', 'SR_TYPE', 'SR_SHORT_CODE', 'OWNER_DEPARTMENT', 'STATUS',
       'CREATED_DATE', 'LAST_MODIFIED_DATE', 'CLOSED_DATE', 'STREET_ADDRESS',
       'CITY', 'STATE', 'ZIP_CODE', 'STREET_NUMBER', 'STREET_DIRECTION',
       'STREET_NAME', 'STREET_TYPE', 'DUPLICATE', 'LEGACY_RECORD',
       'LEGACY_SR_NUMBER', 'PARENT_SR_NUMBER', 'COMMUNITY_AREA', 'WARD',
       'ELECTRICAL_DISTRICT', 'ELECTRICITY_GRID', 'POLICE_SECTOR',
       'POLICE_DISTRICT', 'POLICE_BEAT', 'PRECINCT',
       'SANITATION_DIVISION_DAYS', 'CREATED_HOUR', 'CREATED_DAY_OF_WEEK',
       'CREATED_MONTH', 'X_COORDINATE', 'Y_COORDINATE', 'LATITUDE',
       'LONGITUDE', 'LOCATION'],
      dtype='object')

In [9]:
# chi_311_filtered = chi_311_filtered[['SR_NUMBER', 'OWNER_DEPARTMENT',
#                                      'STATUS', 'CREATED_DATE', 'CLOSED_DATE', 'DUPLICATE',
#                                      'LEGACY_RECORD', 'LEGACY_SR_NUMBER', 'PARENT_SR_NUMBER',
#                                      'WARD', 'CREATED_HOUR', 'CREATED_DAY_OF_WEEK',
#                                      'CREATED_MONTH']]
chi_311_filtered = chi_311_filtered[['SR_NUMBER', 'SR_TYPE',
                                     'STATUS', 'CREATED_DATE', 'CLOSED_DATE', 'DUPLICATE',
                                     'LEGACY_RECORD', 'LEGACY_SR_NUMBER', 'PARENT_SR_NUMBER',
                                     'WARD', 'CREATED_HOUR', 'CREATED_DAY_OF_WEEK',
                                     'CREATED_MONTH']]

In [10]:
chi_311_filtered.shape

(2702596, 13)

Filter out legacy records:

In [11]:
legacy = chi_311_filtered[chi_311_filtered['LEGACY_RECORD'] == True]

In [12]:
max(legacy['CREATED_DATE'])

'12/18/2018 12:47:28 AM'

In [13]:
legacy.shape

(421791, 13)

In [14]:
chi_311_filtered = chi_311_filtered[chi_311_filtered['LEGACY_RECORD'] == False]
chi_311_filtered = chi_311_filtered.drop(columns=['LEGACY_RECORD', 'LEGACY_SR_NUMBER'])

In [15]:
chi_311_filtered.shape

(2280805, 11)

Filter out rows with no community area

In [16]:
chi_311_filtered[chi_311_filtered['WARD'].isna()].shape

(26655, 11)

In [17]:
chi_311_filtered = chi_311_filtered[chi_311_filtered['WARD'].notna()]

In [18]:
chi_311_filtered.shape

(2254150, 11)

Add columns with number of children a given request has

In [19]:
parent_groups = pd.DataFrame(chi_311_filtered['PARENT_SR_NUMBER'].value_counts())
parent_groups = parent_groups.rename(columns={'PARENT_SR_NUMBER': 'NUM_CHILDREN'})

In [20]:
parent_groups

,NUM_CHILDREN
SR20-04442269,162
SR19-01257688,140
SR19-02130163,90
SR19-02130153,88
SR19-02130133,87
...,...
SR19-01013778,1
SR19-02295254,1
SR20-03980483,1
SR19-02298375,1


In [21]:
chi_311_filtered = chi_311_filtered.merge(parent_groups, how='left', left_on='SR_NUMBER',
                                          right_index=True)

In [22]:
len(chi_311_filtered) - chi_311_filtered['NUM_CHILDREN'].isna().sum()

126668

In [23]:
chi_311_filtered = chi_311_filtered[chi_311_filtered['DUPLICATE'] == False]
chi_311_filtered = chi_311_filtered.drop(columns=['DUPLICATE', 'PARENT_SR_NUMBER'])

In [24]:
chi_311_filtered.shape

(1991936, 10)

Make dummy columns.

In [25]:
# chi_311_filtered = pd.get_dummies(chi_311_filtered, columns=['WARD', 'CREATED_HOUR',
#                                           'CREATED_DAY_OF_WEEK', 'CREATED_MONTH', 'OWNER_DEPARTMENT'])

chi_311_filtered = pd.get_dummies(chi_311_filtered, columns=['WARD', 'CREATED_HOUR',
                                          'CREATED_DAY_OF_WEEK', 'CREATED_MONTH', 'SR_TYPE'])

In [26]:
chi_311_filtered.shape

(1991936, 193)

Filtered out complaints resolved in very short period of time -
NEED TO DECIDE WHICH COMPLAINTS TO FILTER OUT!!; do we want to filter out request types with more than a certain % fulfilled in less than X minutes?

In [27]:
chi_311_filtered['CREATED_DATE'] = pd.to_datetime(chi_311_filtered['CREATED_DATE'],
                                                  format='%m/%d/%Y %I:%M:%S %p')
chi_311_filtered['CLOSED_DATE'] = pd.to_datetime(chi_311_filtered['CLOSED_DATE'],
                                                format='%m/%d/%Y %I:%M:%S %p')

In [28]:
chi_311_filtered['time_to_close'] = chi_311_filtered['CLOSED_DATE'] - \
                                        chi_311_filtered['CREATED_DATE']

In [29]:
chi_311_filtered['time_to_close_sec'] = chi_311_filtered['time_to_close'].dt.total_seconds()

In [30]:
chi_311_filtered = chi_311_filtered.drop(columns=['time_to_close'])

In [31]:
# Filter out requests resolved in less than 10 minute
chi_311_filtered = chi_311_filtered[chi_311_filtered['time_to_close_sec'] >= 600]

In [32]:
chi_311_filtered.shape

(1059643, 194)

Pickle filtered file:

In [33]:
chi_311_filtered.to_pickle("../pickle_files/chi_311.pkl")